In [33]:
import warnings

import pymongo
import requests

warnings.filterwarnings(action='ignore')

In [34]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/')  # MongoDB에 연결한다.
starbucks = mongoConnection['starbucks']  # starbucks 데이터베이스를 생성한다.
sido = starbucks['sido']
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
# for s in sidoList:
#     print(s)

In [35]:
for s in sidoList:
    print(s['sido_nm'], end=' ')

서울 경기 광주 대구 대전 부산 울산 인천 강원 경남 경북 전남 전북 충남 충북 제주 세종 

In [36]:
sidoName = input('구군 코드를 검색하려는 시도 이름을 입력하세요.(예: 서울): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당되는 도큐먼트 1건을 얻어온다.
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
print(sido_cd)
print(sido_cd['sido_cd'])

request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do',
                        data={'sido_cd': sido_cd['sido_cd']}).json()
gugunList = []
for r in request['list']:
    d = {'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r['gugun_nm'].strip()}
    # print(d)
    gugunList.append(d)
print(gugunList)

gugun = starbucks['gugun']
gugun.delete_many({})
for g in gugunList:
    gugun.insert_one(g)

{'sido_cd': '01', 'sido_nm': '서울'}
01
[{'gugun_cd': '0101', 'gugun_nm': '강남구'}, {'gugun_cd': '0120', 'gugun_nm': '강동구'}, {'gugun_cd': '0102', 'gugun_nm': '강북구'}, {'gugun_cd': '0103', 'gugun_nm': '강서구'}, {'gugun_cd': '0104', 'gugun_nm': '관악구'}, {'gugun_cd': '0105', 'gugun_nm': '광진구'}, {'gugun_cd': '0121', 'gugun_nm': '구로구'}, {'gugun_cd': '0106', 'gugun_nm': '금천구'}, {'gugun_cd': '0107', 'gugun_nm': '노원구'}, {'gugun_cd': '0108', 'gugun_nm': '도봉구'}, {'gugun_cd': '0122', 'gugun_nm': '동대문구'}, {'gugun_cd': '0109', 'gugun_nm': '동작구'}, {'gugun_cd': '0110', 'gugun_nm': '마포구'}, {'gugun_cd': '0111', 'gugun_nm': '서대문구'}, {'gugun_cd': '0112', 'gugun_nm': '서초구'}, {'gugun_cd': '0123', 'gugun_nm': '성동구'}, {'gugun_cd': '0113', 'gugun_nm': '성북구'}, {'gugun_cd': '0114', 'gugun_nm': '송파구'}, {'gugun_cd': '0115', 'gugun_nm': '양천구'}, {'gugun_cd': '0116', 'gugun_nm': '영등포구'}, {'gugun_cd': '0124', 'gugun_nm': '용산구'}, {'gugun_cd': '0117', 'gugun_nm': '은평구'}, {'gugun_cd': '0118', 'gugun_nm': '종로구'}, {'gugun_cd': '0

시도 이름에 따른 전체 구군을 컬렉션에 도큐먼트로 저장하기

In [37]:
gugun = starbucks['gugun'] # gugun 컬렉션을 만든다.
gugun.delete_many({}) # gugun 컬렉션에 저장된 모든 데이터를 제거한다.

# starbucks db의 sido 컬렉션에 저장된 도큐먼트 개수만큼 반복하며 모든 구군을 gugun 컬렉션에 도큐먼트로 저장한다.
gugunList = []
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
for s in sidoList:
    request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', data={
        'sido_cd': s['sido_cd']
    }).json()
    for r in request['list']:
        # 세종 None 처리
        if r['gugun_nm'] is None:
            r['gugun_nm'] = ''
        d = {'sido_cd': s['sido_cd'].strip(), 'sido_nm': s['sido_nm'].strip(),
             'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r['gugun_nm'].strip()}
        gugunList.append(d)

gugunAll = starbucks['gugunAll']
gugunAll.delete_many({})

for g in gugunList:
    gugun.insert_one(g)